In [275]:
import pandas as pd
import numpy as np
from typing import TypeAlias

import warnings

from sklearn.exceptions import ConvergenceWarning
from statsmodels.tsa.stattools import ValueWarning

warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=ValueWarning)
warnings.filterwarnings('ignore',  category=ConvergenceWarning)

true_date_format = "%y/%m/%d"

ndarrry : TypeAlias = np.ndarray
DataFrame : TypeAlias = pd.DataFrame
Series : TypeAlias = pd.Series

# USD DataFrame

In [276]:
df_usd= pd.read_csv("dol.csv")


df_usd.columns = [
    "Open",
    "Low",
    "High",
    "Close",
    "change",
    "change%",
    "date[gregory]",
    "date[jalali]",
]
true_hirarchy = [
    "date[gregory]",
    "date[jalali]",
    "Open",
    "Low",
    "High",
    "Close",
    "change",
    "change%",
]
pd.to_datetime(df_usd['date[gregory]'], format="ISO8601",  yearfirst=True)

df_usd = df_usd[true_hirarchy]
df_usd.set_index('date[gregory]', inplace=True)

df_usd.drop("change%",axis=1, inplace=True)


df_usd.head()

,date[jalali],Open,Low,High,Close,change
date[gregory],,,,,,
2025/12/09,1404/09/18,"1,259,750","1,243,100","1,266,550","1,248,700",14100
2025/12/08,1404/09/17,"1,240,450","1,240,150","1,263,050","1,262,800",22750
2025/12/07,1404/09/16,"1,221,250","1,221,150","1,240,050","1,240,050",20100
2025/12/06,1404/09/15,"1,197,250","1,197,150","1,222,050","1,219,950",22900
2025/12/04,1404/09/13,"1,203,250","1,195,150","1,206,050","1,197,050",7500


In [277]:
for col in df_usd.columns:
    print(col," => " ,df_usd[col].dtype)

date[jalali]  =>  object
Open  =>  object
Low  =>  object
High  =>  object
Close  =>  object
change  =>  object


In [278]:
for col in df_usd.columns[1:]:
    print(col)
    if col == "change": 
        continue
    else:
        df_usd[col] = df_usd[col].map(lambda x : x.replace(",", "")).astype(np.float32)

df_usd.head()

# "".replace("", "_")

Open
Low
High
Close
change


,date[jalali],Open,Low,High,Close,change
date[gregory],,,,,,
2025/12/09,1404/09/18,1259750.0,1243100.0,1266550.0,1248700.0,14100
2025/12/08,1404/09/17,1240450.0,1240150.0,1263050.0,1262800.0,22750
2025/12/07,1404/09/16,1221250.0,1221150.0,1240050.0,1240050.0,20100
2025/12/06,1404/09/15,1197250.0,1197150.0,1222050.0,1219950.0,22900
2025/12/04,1404/09/13,1203250.0,1195150.0,1206050.0,1197050.0,7500


# Gold-18 DataFrame

In [279]:
df_gold= pd.read_csv("gold.csv")


df_gold.columns = [
    "Open",
    "Low",
    "High",
    "Close",
    "change",
    "change%",
    "date[gregory]",
    "date[jalali]",
]
true_hirarchy = [
    "date[gregory]",
    "date[jalali]",
    "Open",
    "Low",
    "High",
    "Close",
    "change",
    "change%",
]
pd.to_datetime(df_gold['date[gregory]'], format="ISO8601",  yearfirst=True)

df_gold = df_gold[true_hirarchy]
df_gold.set_index('date[gregory]', inplace=True)
df_gold.drop("change%", axis=1, inplace=True)


df_gold.head()

,date[jalali],Open,Low,High,Close,change
date[gregory],,,,,,
2025/12/09,1404/09/18,"128,099,000","125,641,000","128,157,000","126,797,000",335000
2025/12/08,1404/09/17,"127,083,000","126,774,000","127,947,000","127,132,000",67000
2025/12/07,1404/09/16,"124,484,000","124,484,000","127,130,000","127,065,000",2569000
2025/12/06,1404/09/15,"122,501,000","122,501,000","124,994,000","124,496,000",2016000
2025/12/04,1404/09/13,"122,940,000","122,143,000","123,194,000","122,480,000",473000


In [280]:
for col in df_gold.columns:
    print(col," => " ,df_gold[col].dtype)

date[jalali]  =>  object
Open  =>  object
Low  =>  object
High  =>  object
Close  =>  object
change  =>  object


In [281]:

for col in df_gold.columns[1:]:
    # print(col)
    if col == "change": 
        continue

    else:
        df_gold[col] = df_gold[col].map(lambda x : x.replace(",", "")).astype(float, copy=True)

df_gold.head()

# "".replace("", "_")

,date[jalali],Open,Low,High,Close,change
date[gregory],,,,,,
2025/12/09,1404/09/18,128099000.0,125641000.0,128157000.0,126797000.0,335000
2025/12/08,1404/09/17,127083000.0,126774000.0,127947000.0,127132000.0,67000
2025/12/07,1404/09/16,124484000.0,124484000.0,127130000.0,127065000.0,2569000
2025/12/06,1404/09/15,122501000.0,122501000.0,124994000.0,124496000.0,2016000
2025/12/04,1404/09/13,122940000.0,122143000.0,123194000.0,122480000.0,473000


# Combination

In [282]:
# df = pd.DataFrame()

df_usd['usd_price'] = df_usd['Close']
df_gold['gold_price'] = df_gold['Close']



df = pd.merge(df_usd['usd_price'], df_gold['gold_price'],  left_index=True, right_index=True, how='outer')

df

,usd_price,gold_price
date[gregory],,
2023/05/30,509580.0,24950000.0
2023/05/31,513330.0,25040000.0
2023/06/01,510890.0,25040000.0
2023/06/03,509250.0,24647000.0
2023/06/06,496350.0,23869000.0
...,...,...
2025/12/04,1197050.0,122480000.0
2025/12/06,1219950.0,124496000.0
2025/12/07,1240050.0,127065000.0


In [283]:
df.dropna(inplace=True)

# Check for Correlation

In [284]:
# Compute returns
df['usd return'] = df['usd_price'].pct_change()
df['gold return'] = df['gold_price'].pct_change()
df.dropna(inplace=True)

In [285]:
df

,usd_price,gold_price,usd return,gold return
date[gregory],,,,
2023/05/31,513330.0,25040000.0,0.007359,0.003607
2023/06/01,510890.0,25040000.0,-0.004753,0.000000
2023/06/03,509250.0,24647000.0,-0.003210,-0.015695
2023/06/06,496350.0,23869000.0,-0.025331,-0.031566
2023/06/07,492900.0,23486000.0,-0.006951,-0.016046
...,...,...,...,...
2025/12/04,1197050.0,122480000.0,-0.006226,-0.003847
2025/12/06,1219950.0,124496000.0,0.019130,0.016460
2025/12/07,1240050.0,127065000.0,0.016476,0.020635


In [286]:
df[['usd return','gold return']].corr(method="pearson")

,usd return,gold return
usd return,1.000000,0.610602
gold return,0.610602,1.000000


In [287]:
df[['usd return','gold return']].corr(method="spearman")


,usd return,gold return
usd return,1.000000,0.646777
gold return,0.646777,1.000000


In [288]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 718 entries, 2023/05/31 to 2025/12/09
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   usd_price    718 non-null    float32
 1   gold_price   718 non-null    float64
 2   usd return   718 non-null    float32
 3   gold return  718 non-null    float64
dtypes: float32(2), float64(2)
memory usage: 22.4+ KB


In [289]:
df['gold return']

date[gregory]
2023/05/31    0.003607
2023/06/01    0.000000
2023/06/03   -0.015695
2023/06/06   -0.031566
2023/06/07   -0.016046
                ...   
2025/12/04   -0.003847
2025/12/06    0.016460
2025/12/07    0.020635
2025/12/08    0.000527
2025/12/09   -0.002635
Name: gold return, Length: 718, dtype: float64

In [290]:
np.log(df['gold return'].abs())

date[gregory]
2023/05/31   -5.624819
2023/06/01        -inf
2023/06/03   -4.154420
2023/06/06   -3.455684
2023/06/07   -4.132301
                ...   
2025/12/04   -5.560462
2025/12/06   -4.106832
2025/12/07   -3.880757
2025/12/08   -7.547761
2025/12/09   -5.938851
Name: gold return, Length: 718, dtype: float64

# Examine causality between two price series 
## Output: 
    Granger (linear) based detection and MLP based nonlinear test

In [ ]:


from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.api import VAR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from scipy.stats import ttest_rel

# ---------- General settings ----------
SIGNIFICANCE = 0.05
MAX_LAG = 7   # The maximum lag we consider in model selection
MLP_HIDDEN = (128,64, )
MLP_EPOCHS = 500
TEST_SIZE_RATIO = 0.3
TS_SPLITS = 5   # For cross-validation

# ---------- Auxiliary functions ----------
def log_returns(series : Series):
    if np.any(series <0):
        return safe_log_returns(series)
    else:
        return np.log(series).diff().dropna() # type: ignore

def safe_log_returns(series: pd.Series):
    series = series[series > 0]  # Only positive values
    return np.log(series).diff().dropna() # type: ignore


def make_stationary(series: Series, max_diff=2):
    # Apply difference until ADF stops (or reaches max_diff)
    s = series.copy().dropna()
    d = 0
    pvalue = adfuller(s)[1]
    while pvalue > 0.05 and d < max_diff:
        s = s.diff().dropna()
        d += 1
        pvalue = adfuller(s)[1]
    return s, d, pvalue

def select_var_lag(df, maxlags=10, ic='aic'):
    model = VAR(df)
    res = model.select_order(maxlags)
    lag = getattr(res, ic)
    if np.isnan(lag):
        # fallback: choose 1
        return 1
    return int(lag)

# ---------- 1) Granger test (linear) ----------
def granger_test_matrix(x, y, maxlag=10, significance=0.05):
    """
    x, y: pandas Series (stationary). بررسی می‌کنیم x -> y و y -> x با انتخاب lag مناسب.
    برمی‌گرداند: dict با keys 'x_causes_y' و 'y_causes_x' شامل (p-value, chosen_lag, boolean)
    """
    df = pd.concat([x, y], axis=1).dropna()
    df.columns = ['x', 'y']
    chosen_lag = select_var_lag(df, maxlags=min(maxlag, len(df)//5))
    # اجرای granger برای هر جهت (statsmodels نیاز به آرایه با [target, cause])
    # جهت x -> y : قالب برای grangercausalitytests باید [y, x]
    maxlag_to_test = max(1, chosen_lag)
    res_xy = grangercausalitytests(df[['y', 'x']], maxlag=maxlag_to_test, verbose=False)
    pvals_xy = [res_xy[l][0]['ssr_ftest'][1] for l in range(1, maxlag_to_test+1)]
    best_p_xy = min(pvals_xy)
    x_causes_y = best_p_xy < significance

    res_yx = grangercausalitytests(df[['x', 'y']], maxlag=maxlag_to_test, verbose=False)
    pvals_yx = [res_yx[l][0]['ssr_ftest'][1] for l in range(1, maxlag_to_test+1)]
    best_p_yx = min(pvals_yx)
    y_causes_x = best_p_yx < significance

    return {
        'x_causes_y': {'best_pvalue': float(best_p_xy), 'chosen_lag': chosen_lag, 'significant': bool(x_causes_y)},
        'y_causes_x': {'best_pvalue': float(best_p_yx), 'chosen_lag': chosen_lag, 'significant': bool(y_causes_x)},
    }

# ---------- 2) MLP-based nonlinear test ----------
def mlp_predictive_test(x, y, lags=5, splits=5):
    """
    ساخت ویژگی: برای هر زمان t، ورودی شامل لَگ‌های x و لَگ‌های y (تا lags) است.
    مقایسه دو مدل:
      - model_base: فقط لَگ‌های target (مثلاً برای بررسی x->y, target=y -> use y lags)
      - model_full: لَگ‌های target + لَگ‌های candidate cause (x lags)
    با cross-validation زمانی (TimeSeriesSplit) و مقایسه MSE ها با t-test زوجی.
    برمی‌گرداند: (mse_base_mean, mse_full_mean, pvalue_paired_ttest)
    """
    # ساخت ماتریسِ تاخیری
    def build_matrix(a, b, lags):
        # a, b pandas Series, هم‌اندازه
        df = pd.concat([a, b], axis=1).dropna()
        df.columns = ['x', 'y']
        rows = []
        for t in range(lags, len(df)):
            row = {}
            # lagged x
            for L in range(1, lags+1):
                row[f'x_lag{L}'] = df['x'].iloc[t-L]
            # lagged y
            for L in range(1, lags+1):
                row[f'y_lag{L}'] = df['y'].iloc[t-L]
            row['y_target'] = df['y'].iloc[t]
            row['x_target'] = df['x'].iloc[t]
            rows.append(row)
        M = pd.DataFrame(rows).dropna()
        return M

    M = build_matrix(x, y, lags)
    if len(M) < splits+2:
        # داده کم؛ لگ را کم کن یا splits را کم کن
        return None

    X_base = M[[f'y_lag{L}' for L in range(1, lags+1)]].values
    X_full = M[[f'x_lag{L}' for L in range(1, lags+1)] + [f'y_lag{L}' for L in range(1, lags+1)]].values
    y_target = M['y_target'].values

    tscv = TimeSeriesSplit(n_splits=splits)
    mses_base = []
    mses_full = []

    for train_idx, test_idx in tscv.split(X_base):
        Xb_tr, Xb_te = X_base[train_idx], X_base[test_idx]
        Xf_tr, Xf_te = X_full[train_idx], X_full[test_idx]
        y_tr, y_te = y_target[train_idx], y_target[test_idx]

        mbase = MLPRegressor(hidden_layer_sizes=MLP_HIDDEN, max_iter=MLP_EPOCHS, random_state=0)
        mfull = MLPRegressor(hidden_layer_sizes=MLP_HIDDEN, max_iter=MLP_EPOCHS, random_state=0)

        mbase.fit(Xb_tr, y_tr)
        mfull.fit(Xf_tr, y_tr)

        pred_b = mbase.predict(Xb_te)
        pred_f = mfull.predict(Xf_te)

        mses_base.append(mean_squared_error(y_te, pred_b))
        mses_full.append(mean_squared_error(y_te, pred_f))

    # paired t-test: آیا MSE_full < MSE_base به طور معنی‌دار؟
    stat, pval = ttest_rel(mses_base, mses_full, alternative='greater') if hasattr(ttest_rel, 'alternative') else ttest_rel(mses_base, mses_full)
    # Note: scipy <1.9 ttest_rel does not support alternative param; we'll compare means if needed
    # fallback: compute p-two-sided and convert by direction
    if stat is None:
        # fallback compute two-sided
        stat, pval = ttest_rel(mses_base, mses_full)
        # if mean(mses_full) < mean(mses_base): we want one-sided p = pval/2
        if np.mean(mses_full) < np.mean(mses_base):
            pval = pval / 2.0
        else:
            pval = 1.0  # no improvement in desired direction

    return {
        'mse_base_mean': float(np.mean(mses_base)),
        'mse_full_mean': float(np.mean(mses_full)),
        'pvalue': float(pval),
        'mses_base': mses_base,
        'mses_full': mses_full,
        'improves': float(np.mean(mses_full)) < float(np.mean(mses_base))
    }

# ---------- Running the entire analysis ----------
def analyze_causality(price_a, price_b, name_a='A', name_b='B'):
    # price_a, price_b: pandas Series indexed by time
    # Convert to logarithmic efficiency
    rA = log_returns(price_a)
    rB = log_returns(price_b)

    # ایستا کردن
    rA_s, dA, pA = make_stationary(rA)
    rB_s, dB, pB = make_stationary(rB)

    print(f"Stationarity check: diffs applied: {name_a} d={dA}, adf_p={pA:.4f}; {name_b} d={dB}, adf_p={pB:.4f}")

    # هم‌اندازه کن و خطوط پاک‌سازی
    common_index = rA_s.index.intersection(rB_s.index)
    rA_s = rA_s.loc[common_index]
    rB_s = rB_s.loc[common_index]

    # 1) Granger test
    print("\n== Granger causality (linear) ==")
    gr = granger_test_matrix(rA_s, rB_s, maxlag=MAX_LAG, significance=SIGNIFICANCE)
    print(f"{name_a} -> {name_b}: p = {gr['x_causes_y']['best_pvalue']:.4f}, chosen_lag = {gr['x_causes_y']['chosen_lag']}, significant={gr['x_causes_y']['significant']}")
    print(f"{name_b} -> {name_a}: p = {gr['y_causes_x']['best_pvalue']:.4f}, chosen_lag = {gr['y_causes_x']['chosen_lag']}, significant={gr['y_causes_x']['significant']}")

    # 2) Nonlinear predictive test (A -> B)
    print("\n== Nonlinear predictive test (MLP) ==")
    LAGS = min(5, max(1, len(rA_s)//20))  # انتخاب معقول لَگ بر اساس طول داده
    print(f"Using lags = {LAGS} for MLP predictive tests (time-series CV={TS_SPLITS})")

    res_A_to_B = mlp_predictive_test(rA_s, rB_s, lags=LAGS, splits=min(TS_SPLITS, max(2, len(rA_s)//10)))
    res_B_to_A = mlp_predictive_test(rB_s, rA_s, lags=LAGS, splits=min(TS_SPLITS, max(2, len(rA_s)//10)))

    if res_A_to_B is None or res_B_to_A is None:
        print("Not enough data for MLP predictive test (increase samples or reduce lags/splits).")
    else:
        print(f"{name_a} -> {name_b}: mse_base={res_A_to_B['mse_base_mean']:.6f}, mse_full={res_A_to_B['mse_full_mean']:.6f}, p={res_A_to_B['pvalue']:.4f}, improves={res_A_to_B['improves']}")
        print(f"{name_b} -> {name_a}: mse_base={res_B_to_A['mse_base_mean']:.6f}, mse_full={res_B_to_A['mse_full_mean']:.6f}, p={res_B_to_A['pvalue']:.4f}, improves={res_B_to_A['improves']}")

    # تصمیم‌گیر ساده
    print("\n== Decision summary ==")
    # prefer nonlinear test if available, else rely on granger
    def decide(dir_lin, dir_nonlin):
        lin = dir_lin['significant']
        nonlin = (dir_nonlin is not None and dir_nonlin['improves'] and dir_nonlin['pvalue'] < SIGNIFICANCE)
        if nonlin and lin:
            return "Strong evidence (linear + nonlinear)"
        if nonlin and not lin:
            return "Evidence (nonlinear) — linear not significant"
        if lin and not nonlin:
            return "Evidence (linear) — nonlinear not significant"
        return "No evidence"

    dec_ab = decide(gr['x_causes_y'], res_A_to_B)
    dec_ba = decide(gr['y_causes_x'], res_B_to_A)
    print(f"{name_a} -> {name_b}: {dec_ab}")
    print(f"{name_b} -> {name_a}: {dec_ba}")

    # Compare the strength of the direction as well.
    score_ab = 0
    score_ba = 0
    if gr['x_causes_y']['significant']: score_ab += 1
    if gr['y_causes_x']['significant']: score_ba += 1
    if res_A_to_B is not None and res_A_to_B['improves']: score_ab += 1
    if res_B_to_A is not None and res_B_to_A['improves']: score_ba += 1

    if score_ab > score_ba:
        print(f"\nOverall: {name_a} is more likely to be the cause of {name_b} (score {score_ab} vs {score_ba}).")
    elif score_ba > score_ab:
        print(f"\nOverall: {name_b} is more likely to be the cause of {name_a} (score {score_ba} vs {score_ab}).")
    else:
        print("\nOverall: Equal or inconclusive evidence.")




In [292]:
print("base on raw price\n")

analyze_causality(
    price_a=df['gold_price'], 
    price_b=df['usd_price'],
    name_a='gold_price', 
    name_b='usd_price')



base on raw price

Stationarity check: diffs applied: gold_price d=0, adf_p=0.0000; usd_price d=0, adf_p=0.0000

== Granger causality (linear) ==
gold_price -> usd_price: p = 0.0000, chosen_lag = 8, significant=True
usd_price -> gold_price: p = 0.0006, chosen_lag = 8, significant=True

== Nonlinear predictive test (MLP) ==
Using lags = 5 for MLP predictive tests (time-series CV=5)
gold_price -> usd_price: mse_base=0.000499, mse_full=0.000463, p=0.5599, improves=True
usd_price -> gold_price: mse_base=0.000614, mse_full=0.000581, p=0.6513, improves=True

== Decision summary ==
gold_price -> usd_price: Evidence (linear) — nonlinear not significant
usd_price -> gold_price: Evidence (linear) — nonlinear not significant

Overall: Equal or inconclusive evidence.


In [293]:
print("base on Returns\n")
analyze_causality(
    price_a=df['gold return'], 
    price_b=df['usd return'], 
    name_a='gold return', 
    name_b='usd return')


base on Returns

Stationarity check: diffs applied: gold return d=0, adf_p=0.0000; usd return d=0, adf_p=0.0000

== Granger causality (linear) ==
gold return -> usd return: p = 0.0155, chosen_lag = 4, significant=True
usd return -> gold return: p = 0.1461, chosen_lag = 4, significant=False

== Nonlinear predictive test (MLP) ==
Using lags = 5 for MLP predictive tests (time-series CV=5)
gold return -> usd return: mse_base=2.360406, mse_full=3.120849, p=0.0335, improves=False
usd return -> gold return: mse_base=2.342822, mse_full=3.245972, p=0.1503, improves=False

== Decision summary ==
gold return -> usd return: Evidence (linear) — nonlinear not significant
usd return -> gold return: No evidence

Overall: gold return is more likely to be the cause of usd return (score 1 vs 0).
